In [14]:
!pip install rtree

In [15]:
!pip install haversine

In [35]:
# Import necessary libraries
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, box
from rtree import index as rt_index
import re
import time
from haversine import haversine, Unit

In [36]:
# 1. Index-building function
def IndexBuilding(file_path):
    poi_data = pd.read_csv(file_path)
    poi_data['geometry'] = poi_data.apply(lambda row: Point(row['wgs_lng'], row['wgs_lat']), axis=1)
    gdf = gpd.GeoDataFrame(poi_data, geometry='geometry')

    idx = rt_index.Index()
    for poi_id, row in gdf.iterrows():
        idx.insert(int(poi_id), (row.geometry.x, row.geometry.y, row.geometry.x, row.geometry.y), obj=row)

    return idx, gdf

In [37]:
# 2. Range query function
def RangeQuery(query_range, type_regex_str, idx, gdf):
    results = []
    regex = re.compile(type_regex_str)

    if isinstance(query_range, tuple) and len(query_range) == 3:  # Circle range
        center_point = (query_range[0], query_range[1])  # (lat, lon)
        radius = query_range[2]  # Radius in kilometers

        for id in idx.intersection((center_point[1] - radius/111, center_point[0] - radius/111,
                                    center_point[1] + radius/111, center_point[0] + radius/111), objects=True):
            row = gdf.iloc[id.id]
            # Ensure type_code is treated as a string
            type_code_str = str(row['type_code'])
            if regex.match(type_code_str):  # Use the string version for regex matching
                # Note: haversine() expects (lat, lon)
                dist = haversine(center_point, (row.geometry.y, row.geometry.x))
                if dist <= radius:
                    results.append(row)
    else:
        raise ValueError("Invalid query_range format. Must be a circle (x, y, radius).")
    return pd.DataFrame(results)


In [38]:
def NNQuery(query_point, type_regex_str, idx, gdf):
    regex = re.compile(type_regex_str)
    nearest_poi = None
    min_dist = float('inf')  # Set to a very high number initially

    for id in idx.intersection((query_point[1] - 0.05, query_point[0] - 0.05, query_point[1] + 0.05, query_point[0] + 0.05), objects=True):
        row = gdf.iloc[id.id]
        # Ensure type_code is treated as a string
        type_code_str = str(row['type_code'])
        if regex.match(type_code_str):
            # Note: haversine() expects (lat, lon)
            dist = haversine(query_point, (row.geometry.y, row.geometry.x))
            if dist < min_dist:
                nearest_poi = row
                min_dist = dist

    return nearest_poi


In [39]:
# 4. Brute-force range query function
def RangeScan(query_range, type_regex_str, file_path):
    poi_data = pd.read_csv(file_path)
    results = []
    regex = re.compile(type_regex_str)
    for _, row in poi_data.iterrows():
        if regex.match(row['type_code']):
            point = Point(row['wgs_lng'], row['wgs_lat'])
            if isinstance(query_range, tuple) and len(query_range) == 2:  # Rectangle range
                if box(query_range[0][0], query_range[0][1], query_range[1][0], query_range[1][1]).contains(point):
                    results.append(row)
            elif isinstance(query_range, tuple) and len(query_range) == 3:  # Circle range
                if point.distance(Point(query_range[0], query_range[1])) <= query_range[2]:
                    results.append(row)
    return pd.DataFrame(results)


In [40]:
# 5. Brute-force nearest neighbor query function
def NNScan(query_point, type_regex_str, file_path):
    poi_data = pd.read_csv(file_path)
    nearest_poi = None
    min_dist = float('inf')
    regex = re.compile(type_regex_str)
    for _, row in poi_data.iterrows():
        if regex.match(row['type_code']):
            dist = Point(query_point[0], query_point[1]).distance(Point(row['wgs_lng'], row['wgs_lat']))
            if dist < min_dist:
                nearest_poi = row
                min_dist = dist
    return nearest_poi

In [41]:
# Loading and building index from the file
index, gdf = IndexBuilding('/content/Assignment2-2012_BIT_POI.csv')


In [42]:
# Check for restaurants within 500 meters of a specific point
sample_point = (39.955, 116.310)  # Adjust as necessary
sample_radius = 0.5  # 500 meters
sample_restaurants = RangeQuery((sample_point[0], sample_point[1], sample_radius), '^5', index, gdf)
print(f"Number of restaurants within {sample_radius * 1000} meters: {len(sample_restaurants)}")

Number of restaurants within 500.0 meters: 36


In [43]:
# Display some of the found restaurants, if any
if len(sample_restaurants) > 0:
    print(sample_restaurants[['name', 'type_code', 'wgs_lat', 'wgs_lng']])

               name  type_code    wgs_lat     wgs_lng
249    参差咖啡（北京魏公村店）      50500  39.955897  116.312532
70             大象空间      50500  39.955902  116.312352
28              桥咖啡      50500  39.957699  116.306435
181   贝果西饼店（韦伯豪家园西）      50800  39.953295  116.312585
259             稻香村      50800  39.953351  116.313283
65      大才子面馆（魏公村店）      50100  39.953606  116.312426
71       风波庄（魏公村分舵）      50100  39.953619  116.312465
260        咕咕派（北外店）      50000  39.953622  116.312385
247      六和烤鸡（魏公村店）      50000  39.953636  116.312961
180  东北骨头庄（韦伯豪家园西北）      50100  39.953643  116.313156
18        北京晋南建梅主食店      50000  39.953668  116.313368
33          浩日沁蒙古餐厅      50100  39.953719  116.312066
178       肥羊王（魏公村店）      50117  39.953719  116.312066
63         九亿（魏公村店）      50100  39.953846  116.313390
27            花舞陕一边      50115  39.955785  116.314059
254           乡村啤酒屋      50100  39.953008  116.310855
253      渝州家厨（魏公村店）      50102  39.953116  116.311681
248        阿曼尼萨汗美食城      501

In [45]:
# Nearest ATM to the Central Building of BIT
nearest_atm = NNQuery((39.958, 116.311), '^1603', index, gdf)
if nearest_atm is not None:
    print("Nearest ATM:", nearest_atm['name'])
else:
    print("No ATM found")


Nearest ATM: 招商银行ＡＴＭ（魏公村路８号院东北）
